In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from sklearn.model_selection import train_test_split
import torch 
import seaborn as sns

In [56]:
np.random.seed(2)

In [57]:
currency_color_map = {
    'AUD': 'pink',    
    'CAD': 'grey',  
    'DKK': 'red',   
    'EUR': 'blue',     
    'JPY': 'black',  
    'NOK': 'orange',   
    'SEK': 'purple',    
    'GBP': 'green',   
    'USD': 'brown'  
}

currency_rename_map = {
    'ad': 'AUD',  
    'AD': 'AUD',
    'cd': 'CAD',  
    'CD': 'CAD',
    'dk': 'DKK',  
    'DK': 'DKK',
    'EU': 'EUR',  
    'eu': 'EUR',
    'jy': 'JPY',  
    'JY': 'JPY',
    'nk': 'NOK',  
    'NK': 'NOK',
    'sw': 'SEK',  
    'SK': 'SEK',
    'uk': 'GBP',  
    'BP': 'GBP',
    'US': 'USD',
    'us': 'USD'
}

In [58]:
def paste_data(scaled_data, data):

    df = pd.DataFrame(scaled_data,columns=[1,2,3,5,10,15,20,30])
    df.insert(0, 'Currency',data['Currency'])
    df.insert(0, 'Date', data['Date'])

    df_melted = df.melt(id_vars=['Date', 'Currency'], 
                    value_vars=[1, 2, 3, 5, 10, 15, 20, 30], 
                    var_name='Maturity', 
                    value_name='SwapRate')
    return df, df_melted

def calc_rmse(org_data, reconstructed_data):
    currency_rmse = {}
    for currency in org_data["Currency"].unique():

        org = org_data[org_data["Currency"] == currency]
        rec = reconstructed_data[reconstructed_data["Currency"] == currency]
    
        rmse = []

        for i in range(len(org)):
            calc = np.sqrt(np.mean(np.square(org.iloc[i, 2:10] - rec.iloc[i, 2:10]))) * 10000
            rmse.append(calc)
        
        currency_rmse[currency] = np.mean(rmse)

    currency_rmse["Average"] = np.mean([currency_rmse[currency] for currency in currency_rmse.keys()])

    return currency_rmse

In [59]:
def train_validation_split(data, test_size):
    xtrain, xval = train_test_split(data, test_size=test_size,random_state=0)
    return xtrain.reset_index(drop=True), xval.reset_index(drop=True)

def scale_data(data):
    data_array = np.array(data.iloc[:,2:].reset_index(drop=True))/100
    return data_array

def uniform_xavier_weight(input_dim, output_dim):
    no_weights = input_dim+output_dim

    lim = np.sqrt(6/no_weights) 
       
    weights = np.random.uniform(-lim, lim, (input_dim, output_dim))

    return weights

def soft_max(x):
        return 1/(1+np.exp(-x))-0.5

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def soft_max_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_loss_derivative(y_true, y_pred):
    return (1/4) * (y_pred - y_true) 

In [60]:
path          = os.getcwd()
data_path     = os.path.join(os.path.dirname(path), 'Data') 

BloombergData = pd.read_csv(data_path + "/BloombergData_Swap_Features.csv")
preData = pd.read_csv(data_path + "/TestData_Swap_Features_pre.csv")
postData = pd.read_csv(data_path + "/TestData_Swap_Features_post.csv")

x_train1, x_val1 = train_validation_split(BloombergData,0.1)

x_train = scale_data(x_train1) 
x_val   = scale_data(x_val1)
x_pre   = scale_data(preData)
x_post  = scale_data(postData)

In [61]:
class AdamOptimizer:
    def __init__(self, parameters, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.parameters = parameters
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon

        self.m = [np.zeros_like(p) for p in parameters] 
        self.v = [np.zeros_like(p) for p in parameters] 
        self.t = 0  

    def step(self, gradients):
        self.t += 1
        for i, param in enumerate(self.parameters):
            grad = gradients[i]

            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad

            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grad ** 2)

            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)

            param -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

In [62]:
def forward(x, W1, W2, W3):
                            # x  is (N,8)
    
    z1 = np.dot(W1, x.T)    # z1 is (2,N), W1 is (2,8), x.T is (8,N)
    
    z2 = np.dot(W2, z1)     # z2 is (2,N), W2 is (2,2), z1 is (2,N)
    a2 = soft_max(z2)       # a2 is (2,N)
    
    z3 = np.dot(W3, a2)     # z3 is (8,N), W3 is (8,2), a2 is (2,N)      
    z3 = z3.T               # z3.T is (N,8)
    
    return z1, z2, a2, z3

In [63]:
def backward(x, y_true, z1, z2, a2, z3, W2, W3):

    grad_output = mse_loss_derivative(y_true, z3)  
    delta_3 = grad_output       
    
    grad_W3 = np.dot(delta_3.T, a2.T) 
    
    delta_2 = np.dot(W3.T, delta_3.T)*soft_max_derivative(z2)
    grad_W2 = np.dot(delta_2, z1.T)                   

    delta_1 = np.dot(W2.T, delta_2)                  
    grad_W1 = np.dot(delta_1, x)                    

    return [grad_W1, grad_W2, grad_W3]

In [64]:
def train_autoencoder(X_train, X_val, num_epochs=1000, batch_size=32, learning_rate=0.001):
    W1 = uniform_xavier_weight(2, 8)  
    W2 = uniform_xavier_weight(2, 2)  
    W3 = uniform_xavier_weight(8, 2)   

    # Initialize Adam 
    optimizer = AdamOptimizer(parameters=[W1, W2, W3], lr=learning_rate)

    losses_list = []
    losses_val_list = []

    # Training loop
    for epoch in range(num_epochs):
        loss_epoch = 0
        x_train_copy = X_train.copy()
        np.random.shuffle(x_train_copy)   
        
        for i in range(0, x_train_copy.shape[0], batch_size):
            
            X_batch = x_train_copy[i:i + batch_size]
            
            z1, z2, a2, y_pred = forward(X_batch, W1, W2, W3)
             
            loss      = mse_loss(X_batch, y_pred)
            gradients = backward(X_batch, X_batch, z1, z2, a2, y_pred, W2, W3)
            
            optimizer.step(gradients)
            loss_epoch += np.sum(loss)
        
        z1, z2, a2, y_pred = forward(x_train_copy, W1, W2, W3)     
        loss      = mse_loss(x_train_copy, y_pred)
        losses_list.append(loss)

        z1, z2, a2, y_pred = forward(X_val, W1, W2, W3)     
        loss_val      = mse_loss(X_val, y_pred)
        losses_val_list.append(loss_val)

        if (epoch + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.8f}, Loss: {loss_epoch:.8f}")


    return W1, W2, W3, losses_list, losses_val_list 

In [ ]:
# num_epochs =  3196
# batch_size = 32    
# learning_rate = 0.0001  

# W1, W2, W3, loss_train, loss_val = train_autoencoder(x_train, x_train, num_epochs=num_epochs, batch_size=batch_size, learning_rate=learning_rate)

# def reconstruct_autoencoder(X, W1, W2, W3):
#     _, _, _, y_pred = forward(X, W1, W2, W3)
#     return y_pred

# X_reconstructed = reconstruct_autoencoder(x_train, W1, W2, W3)